In [1]:
%load_ext nb_black
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))

<IPython.core.display.Javascript object>

In [5]:
import pandas as pd
import numpy as np
from typing import Tuple, Union, Dict, List

<IPython.core.display.Javascript object>

In [3]:
training_data = pd.DataFrame(
    [
        ["Green", 3, "Apple"],
        ["Yellow", 3, "Apple"],
        ["Red", 1, "Grape"],
        ["Red", 1, "Grape"],
        ["Yellow", 3, "Lemon"],
    ],
    columns=["color", "diameter", "label"],
)

training_data

,color,diameter,label
0,Green,3,Apple
1,Yellow,3,Apple
2,Red,1,Grape
3,Red,1,Grape
4,Yellow,3,Lemon


<IPython.core.display.Javascript object>

In [ ]:
# Use color, diameter, to choose which label
# Train a tree

# Find questions that has maximum information gain

# Find a way to measure the (im)purity of the split data

In [61]:
class DecisionNode:
    def __init__(self, question, true_node, false_node):
        self.question = question
        self.true_node = true_node
        self.false_node = false_node


class Leaf:
    """Just want the class probabilities"""

    def __init__(self, rows):
        self.class_probs = self.find_class_probabilities(rows)

    def find_class_probabilities(self, rows: pd.DataFrame) -> Dict:
        n = len(rows)
        clas_prob = rows.groupby("label").agg(**{"prob": ("label", "count")}) / n
        return clas_prob.to_dict()

    def __repr__(self) -> str:
        return str(self.class_probs)


class Question:
    def __init__(self, feature: str, value: Union[str, float, int]):
        self.feature = feature
        self.value = value
        self.is_numeric_feature = self.is_numeric(value)

    @staticmethod
    def is_numeric(value):
        """Test if a value is numeric."""
        return isinstance(value, int) or isinstance(value, float)

    def ask(self, rows: pd.DataFrame) -> pd.Series:
        if self.is_numeric_feature:
            answer = rows[self.feature] >= self.value
        else:
            answer = rows[self.feature] == self.value

        return answer

    def __repr__(self) -> str:
        if self.is_numeric_feature:
            return f"Is {self.feature} >= {self.value}?"
        else:
            return f"Is {self.feature} == {self.value}?"


# At every node I want to find the best question
class DecisionTree:
    def __init__(self, features):
        self.features = features

    @staticmethod
    def gini(rows: pd.DataFrame) -> pd.DataFrame:
        n = len(rows)
        clas_prob = rows.groupby("label").agg(**{"prob": ("label", "count")}) / n
        return 1 - np.sum(clas_prob["prob"] ** 2)

    def weighted_average_gini(self, true_values, false_values):
        n_true = len(true_values)
        n_false = len(false_values)
        n = n_true + n_false
        weighted_gini = (n_true / n) * self.gini(true_values) + (
            n_false / n
        ) * self.gini(false_values)
        return weighted_gini

    def find_unique_values(self, series: pd.Series) -> List:
        return series.drop_duplicates().to_list()

    def find_best_question(self, rows: pd.DataFrame) -> Tuple[Question, float]:
        # For a data set, what is the bets question I can ask to split the data
        # for every feature I want to create a bunch of questions
        # I want to find the question which reduces the gini the most
        # #
        impurity_before_split = self.gini(rows)

        best_question = None
        best_gain = 0

        for feat in self.features:
            possible_vals = self.find_unique_values(rows[feat])
            for val in possible_vals:
                q = Question(feature=feat, value=val)
                is_cond_true = q.ask(rows=rows)
                true_rows, false_rows = rows[is_cond_true], rows[~is_cond_true]
                # get weighted average gini
                impurity_after_split = self.weighted_average_gini(true_rows, false_rows)
                # find the information gain from asking the question
                info_gain = impurity_before_split - impurity_after_split
                if info_gain >= best_gain:
                    best_question = q
                    best_gain = info_gain

        return best_question, best_gain

    def split(
        self, rows: pd.DataFrame, question: Question
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Return the data for the left and right nodes """

        is_cond_true = question.ask(rows=rows)
        true_rows, false_rows = rows[is_cond_true], rows[~is_cond_true]

        return true_rows, false_rows

    def build_tree(self, rows: pd.DataFrame) -> DecisionNode:
        question_, gain_ = self.find_best_question(rows)

        if gain_ == 0:
            return Leaf(rows=rows)

        true_rows, false_rows = self.split(rows=rows, question=question_)

        return DecisionNode(
            question=question_,
            true_node=self.build_tree(true_rows),
            false_node=self.build_tree(false_rows),
        )

    def fit(rows: pd.DataFrame) -> None:
        pass

<IPython.core.display.Javascript object>

In [10]:
q_eg = Question(feature="color", value="Green")
q_eg.ask(training_data)

0     True
1    False
2    False
3    False
4    False
Name: color, dtype: bool

<IPython.core.display.Javascript object>

In [62]:
mod = DecisionTree(features=["color", "diameter"])

<IPython.core.display.Javascript object>

In [63]:
tree = mod.build_tree(training_data)

<IPython.core.display.Javascript object>

In [37]:
tree

<IPython.core.display.Javascript object>

In [50]:
l = Leaf(rows=training_data)

<IPython.core.display.Javascript object>

In [52]:
print(l)

{'prob': {'Apple': 0.4, 'Grape': 0.4, 'Lemon': 0.2}}


<IPython.core.display.Javascript object>

In [132]:
def print_tree(node: Union[DecisionNode, Leaf], level=0) -> None:
    prefix = " " * 2 ** level + "|__" if level > 0 else ""

    if isinstance(node, Leaf):
        print(prefix, node)
        return

    print(prefix, node.question)

    print_tree(node.true_node, level=level + 1)
    print_tree(node.false_node, level=level + 1)


def predict(node, row: pd.DataFrame, level=0):
    prefix = " " * 2 ** level + "|__" if level > 0 else ""

    if isinstance(node, Leaf):
        return node.class_probs
    print(prefix, node.question)
    is_cond_true = node.question.ask(row)
    print(prefix, is_cond_true.values)
    if is_cond_true.values[0]:
        return predict(node=node.true_node, row=row)
    else:
        return predict(node=node.false_node, row=row)

<IPython.core.display.Javascript object>

In [108]:
print_tree(tree)

 Is diameter >= 3?
  |__ Is color == Yellow?
    |__ {'prob': {'Apple': 0.5, 'Lemon': 0.5}}
    |__ {'prob': {'Apple': 1.0}}
  |__ {'prob': {'Grape': 1.0}}


<IPython.core.display.Javascript object>

In [76]:
mod = DecisionTree(features=["color", "diameter"])
tree = mod.build_tree(training_data)

<IPython.core.display.Javascript object>

In [133]:
testing_data = pd.DataFrame(
    [
        ["Green", 3, "Apple"],
        ["Yellow", 4, "Apple"],
        ["Red", 2, "Grape"],
        ["Red", 1, "Grape"],
        ["Yellow", 3, "Lemon"],
    ],
    columns=["color", "diameter", "label"],
)

predict(tree, testing_data[3:4])

 Is diameter >= 3?
 [False]


{'prob': {'Grape': 1.0}}

<IPython.core.display.Javascript object>

In [104]:
tree.false_node

{'prob': {'Grape': 1.0}}

<IPython.core.display.Javascript object>